지금까지 k-최근접 이웃, 선형 회귀, 릿지, 라쏘, 다항 회귀, 로지스틱 회귀, 확률적 경사 하강법, 결정 트리 모델 등을 배웠다. 근데 이중에서 대체로 성능이 좋은 알고리즘을 선택하라고 하면 무엇을 선택할 수 있을까?

우리가 다루었던 데이터를 되돌아보자.

4장 까지는 생선의 길이, 높이, 무게 등을 데이터로 사용했다. 이 데이터는 CSV 파일에 정리되어 있었고 와인 데이터도 마찬가지다. 이런 형태의 데이터를 **'정형 데이터(structured data)**'라고 부른다. 즉, 어떤 구조로 되어 있다는 의미이다. 이런 데이터는 CSV, 데이터베이스, 엑셀에 저장하기 쉽다. 이와 반대되는 데이터를 **'비정형 데이터(unstructured data)'**라고 부른다. 텍스트 데이터, 사진, 음악 등이 비정형 데이터에 포함된다.

지금까지 배운 머신러닝 알고리즘은 정형 테이터에 잘 맞는다. 그중에 정형 테이터를 다르는 데 가장 뛰어난 성과를 내는 알고리즘이 **'앙상블 함수(ensemble learning)'**이다. 이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다. 비정형 데이터에는 이후에 배울 **'신경망 알고리즘'**을 주로 쓴다.

**'랜덤 포레스트(Random Forest)'**는 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용된다. 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다. 이제 랜덤 포레스트가 어떻게 숲을 만드는지 알아보자.

랜덤 포레스트는 우리가 입력한 **훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다**. 이때 한 샘플이 중복되어 추출될 수도 있다. 이렇게 만들어진 샘플을 **'부트스트랩 샘플(bootstrap sample)'**이라고 부른다. 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.

* **'부트스트랩'**이란 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식이다.

또 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이중에서 최선의 분할을 찾는다. **RandomForestClassifier 클래스**는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다. 회귀 모델인 RandomForestRegressor 클래스는 전체 특성을 사용한다.

사이킷런의 랜덤 포레스트는 기본적으로 **100개의 결정 트리**를 이런 방식으로 훈련한다. 그다음 분류일 때는 각 트리의 **클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측**으로 삼는다. 회귀일 때는 각 트리의 예측을 평균한다.

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

사이킷런의 RandomForestClassifier 클래스를 화이트 와인 분류 문제에 적용해 보자.

데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나눈다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

cross_validate() 함수를 이용해 교차 검증을 수행해 보자. RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 **n_jobs 매개변수를 -1로 지정**하는게 좋다. cross_validate() 함수 역시 마찬가지다. **return_train_score 매개변수**를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환한다. 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하기 좋다..

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9973541965122431 0.8905151032797809


출력 결과를 보니 과대적합인 것으로 보인다. 알고리즘을 조사하는 것이 목적이므로 매개변수를 더 조정하지 않기로 하자.

랜덤 포레스트는 결정 트리의 앙상블이기 때문에 **DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다**. 그리고 결정 트리의 장점인 특성 **중요도를 계산**할 수 있다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.

중요도를 출력해보자.

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


1절 결정 트리의 특성 중요도는 다음과 같다

* [0.12345626 0.86862934 0.0079144]

위와 비교해보면 당도의 중요도가 감소하고 나머지 특성의 중요도가 조금 상승했다. 이는 **특성의 일부를 랜덤하게 선택해 결정 트리를 훈련**해기 때문이다. 따라서 하나의 특성에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 **과대적합을 줄이고 일반화 성능을 높이는데 도움**이 된다.

RandomForestClassifier 클래스에는 자체적으로 모델을 평가하는 함수를 얻을 수 있다. 램덤 포레스트는 부트스트랩 샘플을 만들어 결정 트리를 훈련한다. 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플을 **'OOB(out of bag) 샘플'**이라고 부른다. 이 남는 샘플을 사용해 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 즉, 검증 샘플의 역할과 비슷하다.

이 점수를 얻으려면 oob_score 매개변수를 True로 지정해야한다. OOB 점수를 출력해보자.

In [ ]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수와 비슷한 결과를 얻었다. OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

**'엑스트라 트리(Extra Trees)'**는 랜덤 포레스트와 매우 비슷하다. 기본적으로 100개의 결정 트리를 훈련하고 결정 트리가 제공하는 대부분의 매개변수를 지원한다. 또한 전체 특성 중에 일부 특성을 랜덤하게 선택해 노드를 분할하는데 사용한다.

랜덤 포레스트와 차이점은 **부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용한다**는 점이다. 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 **무작위로 분할**한다. 2절에서 DecisionTreeClassifier 클래스의 splitter 매개변수를 'random'으로 지정할 수 있다고 배웠다. 이 결정 트리가 엑스트라 트리가 사용하는 결정 트리이다.

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능은 낮아진다. 하지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높일 수 있다. 사이킷런의 **ExtraTreesClassifier 클래스**로 교차 검증 점수를 확인해보자.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었다. 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않지만 보통 엑스트라 트리가 무작위성이 더 크기 때문에 **랜덤 포레스트보다 많은 결정 트리를 훈련해야한다**. 하지만 랜덤하게 노드를 분할하여 **계산 속도는 더 빠르다**.

* 결정 트리는 최적의 분할을 찾는 데 시간을 많이 소모한다.

엑스트라 트리도 랜덤 포레스트와 같이 **특성 중요도를 제공**한다.

In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


**'그레이디언트 부스팅(gradient boosting)'**은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다. 사이킷런의 **GradientBoostingClassifier**는 기본적으로 깊이가 3인 결정 트리 100개를 사용한다. 깊이가 얕은 결정 트리를 이용해 과대적합에 강하고 높은 일반화 성능을 기대할 수 있다.

그레이디언트 부스팅은 **경사 하강법을 사용**해 트리를 앙상블에 추가한다. 분류에서는 로지스틱 손실 함수, 회귀에서는 평균 제곱 오차 함수를 사용한다. 

경사 하강법은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정으로 설명했다. 이때 가장 낮은 곳을 찾아 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것이다. 그레이디언트 부스팅은 **결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동**한다. 경사 하강법은 손실 함수의 낮은 곳으로 천천히 이동해야된다 했었다. 이 때문에 우리는 **깊이가 얕은 트리를 사용**하게 된다. 또 **학습률 매개변수**로 속도를 조절한다.

GradientBoostingClassifier 클래스를 사용해 교차 검증 점수를 확인해보자.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


거의 과대적합이 되지 않았다. 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다. 학습률을 증가시키고 트리의 개수를 늘리면 성능이 향상될 수 있다.

In [ ]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


학습률 learning_rate의 기본갑은 0.1이다.

그레이디언트 부스팅 역시 **특성 중요도를 제공**한다.

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


GradientBoostingClassifier 클래스에는 트리 훈련에 사용할 훈련 세트의 비율을 정하는 **subsample 매개변수**가 있다. 기본값은 1.0으로 전체 훈련 세트를 사용하지만 1부다 작으면 훈련 세트의 일부를 사용한다.

일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 **조금 더 높은 성능을 얻을 수 있다**. 하지만 순서대로 트리를 추가하여 **훈련 속도가 느리다**. 순서대로 트리를 추가하기 때문에 n_jobs 매개변수가 없다.

**'히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)'**은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높다. 히스토그램 기반 그레이디언트 부스팅은 **입력 특성을 256개의 구간으로 나누기 때문에 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다**. 또한 256개의 구간 중에 하나를 떼어 놓고 누락된 값을 위해서 사용한다. 

사이킷런의 히소토그램 기반 그레이디언트 부스팅 클래스는 HistGradientBoostingClassifier이다. HistGradientBoostingClassifier에는 트리의 개수를 지정하는데 n_estimators 대신에 **max_iter**를 사용한다.

HistGradientBoostingClassifier 클래스는 아직 테스트 과정에 있다. 따라서 sklearn.experimental 패키지 아래에 **enable_hist_gradient_boosting 모듈**을 임포트해야 한다.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하고 그레이디언트 부스팅보다 조금 높은 성능을 제공한다.

특성 중요도를 계산하기 위해서는 **permutation_importance() 함수**를 사용해야한다. 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다. 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있다.

n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정한다. 기본값은 5이다.

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 **특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)**를 담고있다.
평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있다. 이번에는 테스트 세트의 특성 중요도를 계산해보자.

In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있다. 이런 분석을 통해 모델을 실전에 투입했을 때 **어떤 특성에 관심을 둘지 예상**할 수 있다.

HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인하자.

In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

사이킷런 말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여러가지있다.

가장 대표적인 라이브러리는 **XGBoost**이다. 이 라이브러리는 사이킷런의 cross_validate() 함수와 함께 사용할 수 있다. XGBoost는 다양한 부스팅 알고리즘을 지원하며 **tree_method 매개변수**를 **'hist'**로 지정하면 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있다.

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


널리 사용되는 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 마이크로소프트에서 만든 **LightGBM**이다.

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


# 정리

핵심 포인트
* 앙상블 학습: 더 좋은 예측 결과를 만들기 위해 **여러 개의 모델을 훈련**하는 머신러닝 알고리즘이다.
* 랜덤 포레스트: 대표적인 결정 트리 기반의 앙상블 학습 방법이다. **부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징**이다.
* 엑스트라 트리: 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않는다. 대신 **랜덤하게 노드를 분할**해 과대적합을 감소시킨다.
* 그레이디언트 부스팅:** 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법**이다. 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있다.
* 히스토그램 기반 그레이디언트 부스팅: **그레이디언트 부스팅의 속도를 개선한 것**이며 안정적인 결과와 높은 성능으로 인기가 높다.

핵심 패키지와 함수

scikit-learn
* RandomForestClassifier: **랜덤 포레스트 분류 클래스**이다. **n_estimators 매개변수**로 앙상블을 구성할 트리의 개수를 지정하며 기본값은 100이다. **criterion 매개변수**는 불순도를 지정하며 기본값은 지니 불순도를 의미하는 'gini'이고 'entropy'를 선택하여 엔트로피 불순도를 사용할 수 있다. **max_depth**는 트리가 성장할 최대 깊이를 지정한다. 기본값은 None이고 리프 노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장한다. **min_samples_split**은 노드를 나누기 위한 최소 샘플 개수이다. 기본값은 2이다. **max_features 매개변수**는 최적의 분할을 위해 탐색할 특성의 개수를 지정한다. 기본값은 auto로 특성 개수의 제곱근이다. **bootstrap 매개변수**는 부트스트랩 샘플을 사용할지 지정한다. 기본값은 True이다. **oob_score**는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정한다. 기본값은 False이다. **n_jobs 매개변수**는 병렬 실행에 사용할 CPU 코어 수를 지정한다.
* ExtraTreesClassifier: **엑스트라 트리 분류 클래스**이다. n_estimators, criterion, max_depth, min_samples_split, max_features 매개변수는 랜덤 포레스트와 동일하다. **bootstrap 매개변수**는 부트스트랩 샘플을 사용할지 지정한다. 기본값은 False이다. **oob_score**는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정한다. 기본값은 False이다. **n_jobs 매개변수**는 병렬 실행에 사용할 CPU 코어 수를 지정한다.
* GradientBoostingClassifier: **그레이디언트 부스팅 분류 클래스**이다. **loss 매개변수**로 손실 함수를 지정하며 기본값은 로지스틱 손실 함수를 의미하는 'deviance'이다. **learning_rate 매개변수**는 트리가 앙상블에 기여하는 정도를 조절한다. 기본값은 0.1이다. **n_estimators 매개변수**는 부스팅 단계를 수행하는 트리의 개수이며 기본값은 100이다. **subsample 매개변수**는 사용할 훈련 세트의 샘플 비율을 지정하며 기본값은 1.0이다. **max_depth 매개변수**는 개별 회귀 트리의 최대 깊이이며 기본값은 3이다.
* HistGradientBoostingClassifier: **히스토그램 기반 그레이디언트 부스팅 분류 클래스**이다. **learning_rate 매개변수**는 학습률 또는 감쇠율이라고 하며 기본값은 0.1이며 1.0이면 감쇠가 전혀 없다. **max_iter**는 부스팅 단계를 수행하는 트리의 개수이며 기본값은 100이다. **max_bins**는 입력 데이터를 나눌 구간의 개수이며 기본값은 255이며 이보다 크게 지정할 수 없다. 여기에 1개의 구간이 누락된 값을 위해 추가된다.